Import modules

In [5]:
import sys
import os
import itertools

if '..' not in sys.path:
    sys.path.append('..')

from src.text_chunk import TextChunk
from src.stylometry_extractor import StylometryExtractor
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

Read raw train texts

In [6]:
def get_data(path):
    data = []

    for i in itertools.count(start=1):
        try:
            text = open(os.path.join(path, 'problem-' + str(i) + '.txt'), 'r').read()
            data.append(TextChunk(text))
        except FileNotFoundError:
            break
    
    return data


train = get_data('../data/train_raw')

Fit a TF-IDF vectorizer to all character fourgrams in train data

In [8]:
split_by = StylometryExtractor.SPECIAL_CHAR

vectorizer = TfidfVectorizer(tokenizer=lambda x, s=split_by: x.split(s),
                             max_features=500,
                             stop_words=stopwords.words('english'))

vectorizer.fit(map(lambda text: text.ngram_string, train))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=500, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',... 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function <lambda> at 0x10dea6f28>, use_idf=True,
        vocabulary=None)

In [11]:
assert len(vectorizer.vocabulary_) == 500

Serialize vectorizer

In [12]:
import dill

with open('../data/char_ngram_vectorizer.pk', 'wb') as f:
    dill.dump(vectorizer, f)